In [ ]:
import os
os.getcwd()
os.chdir('/mnt/home/agoldmaier/repos/master-thesis-dissaggregator')

In [ ]:
# %%
import datetime
from datetime import timedelta

import holidays
import pandas as pd

from src import logger
from src.configs.data import *
from src.data_access.local_reader import *
from src.data_processing.consumption import *
from src.data_processing.temperature import *
from src.pipeline.pipe_applications import *
from src.utils.utils import *

In [ ]:
# %%
def get_timezone(alpha2code):

    """
      getting timezone of country in Europe
    """

    timezonemap = {

        'AT': 'Europe/Berlin', 'BE': 'Europe/Berlin', 'BG': 'Europe/Sofia', 'BA' : 'Europe/Sarajevo', 'CH': 'Europe/Zurich', 'CY': 'Europe/Sofia', 
        
        'CZ': 'Europe/Sofia', 'DE': 'Europe/Berlin',

        'DK': 'Europe/Berlin', 'EE': 'Europe/Sofia', 'GB': 'Europe/London', 'GR': 'Europe/Sofia', 'ES': 'Europe/Berlin', 'FI': 'Europe/Sofia', 
        
        'FR': 'Europe/Berlin', 'HR': 'Europe/Berlin', 'HU': 'Europe/Berlin', 'IE': 'Europe/London', 'IS': 'Atlantic/Reykjavik', 'IT': 'Europe/Berlin', 'LT': 'Europe/Sofia', 
        
        'LU': 'Europe/Berlin', 'LV': 'Europe/Sofia', 'ME': 'Europe/Podgorica', 'MK': 'Europe/Skopje', 'MT': 'Europe/Sofia', 'NL': 'Europe/Berlin', 'NO': 'Europe/Oslo', 'PL': 'Europe/Berlin', 'PT': 'Europe/London', 
        
        'RO': 'Europe/Berlin', 'RS': 'Europe/Belgrade', 'SE': 'Europe/Berlin', 'SI': 'Europe/Berlin', 'SK': 'Europe/Berlin', 'UK': 'Europe/London'

    }


    return timezonemap.get(alpha2code)


def make_year_index(year: int, freq: str, tz):
    year_start = pd.Timestamp(str(year), tz="UTC")
    year_end = pd.Timestamp(str(year + 1), tz="UTC")

    return (
        pd.date_range(start=year_start, end=year_end, freq=freq)[:-1]
        .tz_convert(tz)
    )

In [ ]:
# %%
state = 'BW'
year = 2022
profile ='H0'

In [ ]:
# %%
def make_year_index(year: int, freq: str, tz):
    year_start = pd.Timestamp(str(year), tz="UTC")
    year_end = pd.Timestamp(str(year + 1), tz="UTC")

    return (
        pd.date_range(start=year_start, end=year_end, freq=freq)[:-1]
        .tz_convert(tz)
    )

In [ ]:
# %%
def load_power_load_profile(profile: str) -> pd.DataFrame:
    """
    Retuns the power load profiles for the given profile.
    DISS: "4.2.5.2 Standardlastprofile" -> Tabelle A.9
    """

    raw_file = f"data/raw/temporal/power_load_profiles/39_VDEW_Strom_Repräsentative_Profile_{profile}.xlsx"
    load_profiles = pd.read_excel(raw_file)

    return load_profiles

In [ ]:
# %%
def Leistung(Tag_Zeit, mask, df, df_SLP):
    u = pd.merge(df[mask], df_SLP[["Hour", Tag_Zeit]], on=["Hour"], how="left")
    v = pd.merge(df, u[["Date", Tag_Zeit]], on=["Date"], how="left")
    v_filled = v.infer_objects(copy=False).fillna(0.0)
    v_filled = v_filled.infer_objects(copy=False)
    return v_filled[Tag_Zeit]


tz = get_timezone("DE")

year_start = pd.Timestamp(str(year), tz="UTC")
year_end = pd.Timestamp(str(year + 1), tz="UTC")

idx = make_year_index(year, "15min", tz)


#idx = pd.date_range(start=str(year), end=str(year + 1), freq="15min")[:-1]

df = (
    pd.DataFrame(data={"Date": idx})
    .assign(Day=lambda x: pd.DatetimeIndex(x["Date"]).date)
    .assign(Hour=lambda x: pd.DatetimeIndex(x["Date"]).time)
    .assign(DayOfYear=lambda x: pd.DatetimeIndex(x["Date"]).dayofyear.astype(int))
)

mask_holidays = []
for i in range(0, len(holidays.DE(state=state, years=year))):
    mask_holidays.append("Null")
    mask_holidays[i] = (
        df["Day"] == [x for x in holidays.DE(state=state, years=year).items()][i][0]
    )

hd = mask_holidays[0]

for i in range(1, len(holidays.DE(state=state, years=year))):
    hd = hd | mask_holidays[i]

df["WD"] = df["Date"].apply(lambda x: x.weekday() < 5) & (~hd)
df["SA"] = df["Date"].apply(lambda x: x.weekday() == 5) & (~hd)
df["SU"] = df["Date"].apply(lambda x: x.weekday() == 6) | hd

mask = df["Day"].isin([datetime.date(year, 12, 24), datetime.date(year, 12, 31)])

df.loc[mask, ["WD", "SU"]] = False
df.loc[mask, "SA"] = True

wiz1 = df.loc[df["Date"] < (str(year) + "-03-21 00:00:00")]
wiz2 = df.loc[df["Date"] >= (str(year) + "-11-01")]

soz = df.loc[
    ((str(year) + "-05-15") <= df["Date"]) & (df["Date"] < (str(year) + "-09-15"))
]
uez1 = df.loc[
    ((str(year) + "-03-21") <= df["Date"]) & (df["Date"] < (str(year) + "-05-15"))
]
uez2 = df.loc[
    ((str(year) + "-09-15") <= df["Date"]) & (df["Date"] <= (str(year) + "-10-31"))
]

df = df.assign(
    WIZ=lambda x: (x.Day.isin(wiz1.Day) | x.Day.isin(wiz2.Day)),
    SOZ=lambda x: x.Day.isin(soz.Day),
    UEZ=lambda x: (x.Day.isin(uez1.Day) | x.Day.isin(uez2.Day)),
)

last_strings = []

In [ ]:
df_load = load_power_load_profile(profile)

df_load.columns = [
            "Hour",
            "SA_WIZ",
            "SU_WIZ",
            "WD_WIZ",
            "SA_SOZ",
            "SU_SOZ",
            "WD_SOZ",
            "SA_UEZ",
            "SU_UEZ",
            "WD_UEZ",
        ]



In [ ]:
df_SLP = df_load[2:97]

In [ ]:
df_SLP.loc[0:len(df_SLP)-1,'Hour'] = list(df_SLP.loc[[len(df_SLP)-1] + list(range(0, len(df_SLP)-1)), 'Hour'])
        

In [ ]:

df_SLP = df_load[1:97]
df_SLP = df_SLP.reset_index()[
    [
        "Hour",
        "SA_WIZ",
        "SU_WIZ",
        "WD_WIZ",
        "SA_SOZ",
        "SU_SOZ",
        "WD_SOZ",
        "SA_UEZ",
        "SU_UEZ",
        "WD_UEZ",
    ]
]